In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import csv


def get_shop_star_rating(shop_url):
    """1店舗のURLから星評価（例:3.56）を取得"""
    try:
        res = requests.get(shop_url)
        soup = BeautifulSoup(res.content, 'html.parser')
        star_elem = soup.select_one('span.rdheader-rating__score-val-dtl')
        star_rating = star_elem.get_text(strip=True) if star_elem else "N/A"
        return star_rating
    except Exception as e:
        print(f"星評価取得エラー({shop_url}): {e}")
        return "N/A"

def get_shop_main_categories(shop_url):
    """
    <span class="linktree__parent-target-text">のみのジャンル名テキストを抽出。リンク・aタグ等は完全に無視！
    """
    try:
        res = requests.get(shop_url)
        soup = BeautifulSoup(res.content, 'html.parser')
        
        # <span class="linktree__parent-target-text">のテキストを全部抽出
        genre_spans = soup.select('span.linktree__parent-target-text')
        main_categories = [span.get_text(strip=True) for span in genre_spans]
        
        # 例: ['焼肉', '牛タン', '居酒屋']
        return ','.join(main_categories) if main_categories else "N/A"
    except Exception as e:
        print(f"ジャンル取得エラー({shop_url}): {e}")
        return "N/A"



# --- 関数1：エリア内の全店舗URLを取得する ---
def get_all_shop_urls_from_area(area_top_url):
    """
    指定されたエリアの店舗一覧を全ページ巡回し，
    全店舗のURLと店名をリストで返す関数
    """
    shop_list = []
    page = 1
    
    while True:
        if page == 1:
            target_url = area_top_url
        else:
            target_url = f"{area_top_url.rstrip('/')}/rstLst/{page}/"
            
        print(f"店舗一覧の{page}ページ目を収集中...")
        
        try:
            response = requests.get(target_url)
            if response.status_code != 200:
                print("-> 店舗一覧の次のページが見つかりませんでした．")
                break
            
            soup = BeautifulSoup(response.content, 'html.parser')
            shop_tags = soup.select('a.list-rst__rst-name-target')
            
            if not shop_tags:
                print("-> このページに店舗情報がありませんでした．")
                break

            for tag in shop_tags:
                shop_list.append({
                    'name': tag.get_text(strip=True),
                    'url': tag['href']
                })

            page += 1
            time.sleep(1)
        except requests.exceptions.RequestException as e:
            print(f"エラーが発生しました: {e}")
            break
            
    return shop_list

# --- 関数2：1店舗の全口コミを取得する ---
def get_all_reviews_from_shop(shop_url):
    """1つの店舗URLから，全ての口コミ本文を取得する"""
    
    shop_reviews = []
    page_num = 1
    review_base_url = f"{shop_url}dtlrvwlst/COND-0/smp1/?smp=1&lc=0&rvw_part=all"

    while True:
        if page_num == 1:
            target_url = review_base_url
        else:
            target_url = f"{review_base_url}&PG={page_num}"
        
        print(f"  -> 口コミの{page_num}ページ目を収集中...")
        
        try:
            response = requests.get(target_url)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')
            review_elements = soup.find_all('div', class_='rvw-item__rvw-comment')

            if not review_elements:
                break

            for review in review_elements:
                shop_reviews.append(review.get_text(strip=True))

            page_num += 1
            time.sleep(1)
        except requests.exceptions.RequestException as e:
            print(f"    口コミ取得中にエラー: {e}")
            break
            
    return shop_reviews

# --- メインの実行処理 ---
if __name__ == '__main__':
    
    # 1. お台場エリアのトップページのURLを指定
    odaiba_area_url = 'https://tabelog.com/tokyo/A1313/A131306/'
    
    # 2. 関数1を呼び出し，お台場の全店舗リストを取得
    print("--- ステップ1：お台場の店舗リスト収集開始 ---")
    all_shops_in_odaiba = get_all_shop_urls_from_area(odaiba_area_url)
    print(f"\n収集完了．お台場エリアで {len(all_shops_in_odaiba)}軒の店舗が見つかりました．\n")
    
    # 3. CSVファイルを用意し，各店舗の口コミを収集・保存
    print("--- ステップ2：各店舗の口コミ収集開始 ---")
    output_filename = 'odaiba_reviews_4.csv'
    
    with open(output_filename, 'w', encoding='utf-8-sig', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['shop_name', 'shop_url', 'category', 'star_rating', 'review_text'])
        
        for shop in all_shops_in_odaiba:
            shop_name = shop['name']
            shop_url = shop['url']
            print(f"\n【{shop_name}】の口コミを取得")
            
            # ★ 追加：星評価とカテゴリ取得
            star_rating = get_shop_star_rating(shop_url)
            category = get_shop_category(shop_url)
            print(f"  -> 星評価: {star_rating}")
            print(f"  -> カテゴリ: {category}")
            
            reviews = get_all_reviews_from_shop(shop_url)
            print(f"  -> {len(reviews)}件の口コミを取得．CSVに書き込みます．")
            
            # 書き込み時に「shop_name, shop_url, category, 星評価, レビュー本文」で書き込む
            for review_text in reviews:
                writer.writerow([shop_name, shop_url, category, star_rating, review_text])
            
            print("  -> 3秒待機")
            time.sleep(3)


    print(f"\n全ての処理が完了．「{output_filename}」")

--- ステップ1：お台場の店舗リスト収集開始 ---
店舗一覧の1ページ目を収集中...
店舗一覧の2ページ目を収集中...
店舗一覧の2ページ目を収集中...
店舗一覧の3ページ目を収集中...
店舗一覧の3ページ目を収集中...
店舗一覧の4ページ目を収集中...
店舗一覧の4ページ目を収集中...
店舗一覧の5ページ目を収集中...
店舗一覧の5ページ目を収集中...
店舗一覧の6ページ目を収集中...
店舗一覧の6ページ目を収集中...
店舗一覧の7ページ目を収集中...
店舗一覧の7ページ目を収集中...
店舗一覧の8ページ目を収集中...
店舗一覧の8ページ目を収集中...
店舗一覧の9ページ目を収集中...
店舗一覧の9ページ目を収集中...
店舗一覧の10ページ目を収集中...
店舗一覧の10ページ目を収集中...
店舗一覧の11ページ目を収集中...
店舗一覧の11ページ目を収集中...
店舗一覧の12ページ目を収集中...
店舗一覧の12ページ目を収集中...
店舗一覧の13ページ目を収集中...
店舗一覧の13ページ目を収集中...
店舗一覧の14ページ目を収集中...
店舗一覧の14ページ目を収集中...
店舗一覧の15ページ目を収集中...
店舗一覧の15ページ目を収集中...
店舗一覧の16ページ目を収集中...
店舗一覧の16ページ目を収集中...
店舗一覧の17ページ目を収集中...
店舗一覧の17ページ目を収集中...
店舗一覧の18ページ目を収集中...
店舗一覧の18ページ目を収集中...
店舗一覧の19ページ目を収集中...
店舗一覧の19ページ目を収集中...
店舗一覧の20ページ目を収集中...
店舗一覧の20ページ目を収集中...
店舗一覧の21ページ目を収集中...
店舗一覧の21ページ目を収集中...
店舗一覧の22ページ目を収集中...
店舗一覧の22ページ目を収集中...
店舗一覧の23ページ目を収集中...
店舗一覧の23ページ目を収集中...
店舗一覧の24ページ目を収集中...
店舗一覧の24ページ目を収集中...
店舗一覧の25ページ目を収集中...
店舗一覧の25ページ目を収集中...
店舗一覧の26ページ目を収集中...
店舗一覧の26ページ目を収集中...
店舗一覧の27ページ目を収集中...
店